CC5213 - Recuperación de Información Multimedia
==

Slides 1.4 - Detección de Bordes
==

In [3]:
import sys
import os
import numpy
import cv2
from PyQt5.QtWidgets import QApplication, QFileDialog
import matplotlib.pyplot as plt

def mostrar_imagen(window_name, imagen):
    MAX_WIDTH = 700
    MAX_HEIGHT = 500
    if imagen.shape[0] > MAX_HEIGHT or imagen.shape[1] > MAX_WIDTH:
        #reducir tamaño
        fh = MAX_HEIGHT / imagen.shape[0]
        fw = MAX_WIDTH / imagen.shape[1]
        escala = min(fh, fw)
        imagen = cv2.resize(imagen, (0,0), fx=escala, fy=escala)
    #mostrar en pantalla
    cv2.imshow(window_name, imagen)

def ui_select_filenames():
    app = QApplication(list());
    options = QFileDialog.Options()
    files, _ = QFileDialog.getOpenFileNames(None, "Imagenes", ".", "Imagenes (*.jpg *.png)", options=options)
    return files

def abrir_imagen(filename):
    imagen_color = cv2.imread(filename, cv2.IMREAD_COLOR)
    if imagen_color is None:
        raise Exception("error abriendo {}".format(filename))
    return imagen_color



def normalizar(imagen, valorAbsoluto = False, min0Max255 = False):
    img = imagen
    if valorAbsoluto:
        img = numpy.abs(imagen)
    if min0Max255:
        img2 = cv2.normalize(img, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        img = img2
    return img

print("Usando OpenCVa {} con Python {}.{}.{}".format(cv2.__version__, sys.version_info.major, sys.version_info.minor, sys.version_info.micro))

Usando OpenCVa 4.4.0 con Python 3.7.1
['/Users/matias/Desktop/Universidad/RIM', '/anaconda3/lib/python37.zip', '/anaconda3/lib/python3.7', '/anaconda3/lib/python3.7/lib-dynload', '', '/anaconda3/lib/python3.7/site-packages', '/anaconda3/lib/python3.7/site-packages/aeosa', '/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/matias/.ipython']


## Ejemplo 1 - Sobel

In [2]:
def ejemplo(filename):
    sobel_threshold = 200
    delta = 5
    imagen_color = abrir_imagen(filename)
    imagen_gris = cv2.cvtColor(imagen_color, cv2.COLOR_BGR2GRAY)
    #calcular filtro de sobel
    sobelX = cv2.Sobel(imagen_gris, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=3)
    sobelY = cv2.Sobel(imagen_gris, ddepth=cv2.CV_32F, dx=0, dy=1, ksize=3)
    #magnitud del gradiente
    magnitud = numpy.sqrt(numpy.square(sobelX) + numpy.square(sobelY) )
    mostrar_imagen("MAGNITUD GRADIENTE", normalizar(magnitud, min0Max255=True))
    retval, bordes = cv2.threshold(magnitud, thresh=sobel_threshold, maxval=255, type=cv2.THRESH_BINARY)
    mostrar_imagen("BORDES", bordes)

filenames = ui_select_filenames()

if len(filenames) > 0:
    for filename in filenames:
        ejemplo(filename)
    cv2.waitKey(30)
    cv2.destroyAllWindows()


print(sobelX)
print(sobelY)
print(magnitud)
print(bordes)

print("FIN")

NameError: name 'sobelX' is not defined

## Ejemplo 2 - Canny

In [ ]:
def ejemplo(filename):
    global canny_threshold_1, canny_threshold_2, delta
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular canny
        frame_canny = cv2.Canny(frame_gris, threshold1=canny_threshold_1, threshold2=canny_threshold_2)
        agregar_texto(frame_canny, "th={}-{}".format(canny_threshold_1, canny_threshold_2))
        mostrar_imagen("CANNY", frame_canny)
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            canny_threshold_1 += delta
        elif key == ord('z'):
            if canny_threshold_1 - delta > 0:
                canny_threshold_1 -= delta
        elif key == ord('s'):
            canny_threshold_2 += delta
        elif key == ord('x'):
            if canny_threshold_2 - delta > 0:
                canny_threshold_2 -= delta
        else:
            print("unknown key '{}' ({})".format(chr(key), key))
    capture.release()
    cv2.destroyAllWindows()

canny_threshold_1 = 51
canny_threshold_2 = 301
delta = 10

filename = ui_select_video()
ejemplo(filename)

print("FIN")

## Ejemplo 3 - Difference of Gaussians (DoG)

In [ ]:
def ejemplo(filename):
    global sigma1, sigma2, threshold
    capture = abrir_video(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        #convertir a gris
        frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mostrar_imagen("VIDEO", frame_gris)
        #calcular DoG
        blur1 = cv2.GaussianBlur(frame_gris, (sigma1, sigma1), 0)
        blur2 = cv2.GaussianBlur(frame_gris, (sigma2, sigma2), 0)
        frame_diff = cv2.subtract(blur1, blur2)
        mostrar_imagen("Diff", normalizar(frame_diff, min0Max255=True))
        threshold = round(threshold, 2)
        th, frame_bin = cv2.threshold(frame_diff, threshold, 255, cv2.THRESH_BINARY)
        agregar_texto(frame_bin, "th={}".format(threshold))
        mostrar_imagen("BIN", normalizar(frame_bin, min0Max255=True))
        #esperar por una tecla
        key = cv2.waitKey(10)
        if key == ord(' '):
            key = cv2.waitKey(0)
        if key == -1:
            continue
        elif key == ord('q') or key == 27:
            break
        elif key == ord('a'):
            threshold += delta
        elif key == ord('z'):
            if threshold - delta > 0:
                threshold -= delta
    capture.release()
    cv2.destroyAllWindows()

sigma1 = 3
sigma2 = 13
threshold = 5
delta = 0.05

filename = ui_select_video()
ejemplo(filename)

print("FIN")